In [1]:
import pandas as pd
import pickle

In [2]:
train_log = pd.read_csv("../data/train_log.csv")
train_label = pd.read_csv("../data/train_label.csv")

test_log = pd.read_csv("../data/test_log.csv")
test_session = pd.read_csv("../data/test_session.csv")

yado = pd.read_csv("../data/yado.csv")

sample_submission = pd.read_csv("../data/sample_submission.csv")

In [3]:
train = pd.read_csv(
    f"../saved_data/e015_add_previous_yado/e015_add_previous_yado_train.csv"
)

# test = pd.read_csv(
#     f"../saved_data/e015_add_previous_yado/e015_add_previous_yado_test.csv"
# )

In [4]:
with open(
    f"../saved_data/e015_add_previous_yado/e015_add_previous_yado_oof.pkl", "rb"
) as f:
    oof = pickle.load(f)

In [5]:
sub = pd.read_csv("../sub/e015_add_previous_yado_auc0.8723_mapk0.6315.csv")

In [6]:
train["oof"] = oof

In [7]:
train_for_oof = train[train["oof"] != 0]

In [8]:
train_for_oof_with_yad = pd.merge(train_for_oof, yado, on="yad_no", how="left")

In [10]:
# trainをoofが0でないものだけに絞る
import numpy as np


def get_oof_pred_df(train: pd.DataFrame, oof: np.ndarray) -> pd.DataFrame:
    train_for_calc_mapk = train.copy()
    train_for_calc_mapk["oof"] = oof
    train_for_calc_mapk = train_for_calc_mapk[train_for_calc_mapk["oof"] != 0]

    # 推論順にsession_idとyad_noを並べる
    oof_pred_yad = (
        train_for_calc_mapk.sort_values(["session_id", "oof"], ascending=False)
        .groupby("session_id")["yad_no"]
        .apply(list)
    ).to_dict()

    # train_labelをoofの計算用に用意
    train_label_for_calc_oof = train_label.copy()

    # train_for_calc_mapkに付与
    train_label_for_calc_oof["pred_yad_no_list"] = train_label_for_calc_oof[
        "session_id"
    ].map(oof_pred_yad)

    # oofが計算されていないsession_idは削除
    train_label_for_calc_oof = train_label_for_calc_oof[
        train_label_for_calc_oof["pred_yad_no_list"].notnull()
    ].reset_index(drop=True)

    # 上位10件に限定
    train_label_for_calc_oof["pred_yad_no_list_top10"] = train_label_for_calc_oof[
        "pred_yad_no_list"
    ].apply(lambda x: x[:10])

    # listをpd.Seriesに変換
    oof_pred_df = train_label_for_calc_oof.set_index("session_id")[
        "pred_yad_no_list_top10"
    ].apply(pd.Series)
    oof_pred_df = oof_pred_df.rename(columns=lambda x: "predict_" + str(x))

    # Nullの箇所はyad_no=0で保管し、全ての値をintに変換する
    # TODO: 埋めるのは0で本当に良いのか考える
    oof_pred_df = oof_pred_df.fillna(0).astype(int)

    return oof_pred_df


oof_pred_df = get_oof_pred_df(train, oof)

In [11]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k for a single actual value.

    Parameters:
    actual : int
        The actual value that is to be predicted
    predicted : list
        A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The average precision at k
    """
    if actual in predicted[:k]:
        return 1.0 / (predicted[:k].index(actual) + 1)
    return 0.0


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k for lists of actual values and predicted values.

    Parameters:
    actual : list
        A list of actual values that are to be predicted
    predicted : list
        A list of lists of predicted elements (order does matter in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns:
    float
        The mean average precision at k
    """
    return sum(apk(a, p, k) for a, p in zip(actual, predicted)) / len(actual)

In [12]:
# MAPK (k=10) として計算
sorted_train_label = (
    train_label[train_label["session_id"].isin(oof_pred_df.index)]
    .sort_values("session_id")["yad_no"]
    .values
)

assert len(sorted_train_label) == len(oof_pred_df)

oof_mapk_score = mapk(
    actual=sorted_train_label,
    predicted=oof_pred_df.sort_index().values.tolist(),
    k=10,
)
oof_mapk_score

0.6315050045951125

In [14]:
train_label

,session_id,yad_no
0,000007603d533d30453cc45d0f3d119f,4101
1,0000ca043ed437a1472c9d1d154eb49b,8253
2,0000d4835cf113316fe447e2f80ba1c8,4863
3,0000fcda1ae1b2f431e55a7075d1f500,1652
4,000104bdffaaad1a1e0a9ebacf585f33,96
...,...,...
288693,ffff2262d38abdeb247ebd591835dcc9,2259
288694,ffff2360540745117193ecadcdc06538,963
288695,ffff7fb4617164b2604aaf51c40bf82d,13719
288696,ffffcd5bc19d62cad5a3815c87818d83,10619


In [19]:
session_id_yad_no_cnt_dict = (
    train_log.groupby("session_id")["yad_no"].apply(len).to_dict()
)
train_label["session_id_yad_no_cnt"] = train_label["session_id"].map(
    session_id_yad_no_cnt_dict
)

In [21]:
train_label

,session_id,yad_no,session_id_yad_no_cnt
0,000007603d533d30453cc45d0f3d119f,4101,1
1,0000ca043ed437a1472c9d1d154eb49b,8253,1
2,0000d4835cf113316fe447e2f80ba1c8,4863,1
3,0000fcda1ae1b2f431e55a7075d1f500,1652,1
4,000104bdffaaad1a1e0a9ebacf585f33,96,2
...,...,...,...
288693,ffff2262d38abdeb247ebd591835dcc9,2259,1
288694,ffff2360540745117193ecadcdc06538,963,1
288695,ffff7fb4617164b2604aaf51c40bf82d,13719,1
288696,ffffcd5bc19d62cad5a3815c87818d83,10619,3


In [23]:
oof_pred_df["session_id_yad_no_cnt"] = oof_pred_df.index.map(session_id_yad_no_cnt_dict)

In [28]:
train_label_session_length_1 = train_label[train_label["session_id_yad_no_cnt"] == 1]
train_label_session_length_over_2 = train_label[
    train_label["session_id_yad_no_cnt"] >= 2
]
train_label_session_length_over_3 = train_label[
    train_label["session_id_yad_no_cnt"] >= 3
]

oof_pred_df_session_length_1 = oof_pred_df[oof_pred_df["session_id_yad_no_cnt"] == 1]
oof_pred_df_session_length_over_2 = oof_pred_df[
    oof_pred_df["session_id_yad_no_cnt"] >= 2
]
oof_pred_df_session_length_over_3 = oof_pred_df[
    oof_pred_df["session_id_yad_no_cnt"] >= 3
]

In [29]:
# session_lengthが1のデータに限定する


def calc_mapk(train_lable: pd.DataFrame, oof_pred_df: pd.DataFrame) -> float:
    sorted_train_label = (
        train_label[train_label["session_id"].isin(oof_pred_df.index)]
        .sort_values("session_id")["yad_no"]
        .values
    )

    assert len(sorted_train_label) == len(oof_pred_df)

    oof_mapk_score = mapk(
        actual=sorted_train_label,
        predicted=oof_pred_df.sort_index().values.tolist(),
        k=10,
    )

    return oof_mapk_score

In [31]:
calc_mapk(train_label, oof_pred_df)

0.6315050045951125

In [30]:
calc_mapk(train_label_session_length_1, oof_pred_df_session_length_1)

0.4823531650354597

In [32]:
calc_mapk(train_label_session_length_over_2, oof_pred_df_session_length_over_2)

0.8958688696516477

In [33]:
calc_mapk(train_label_session_length_over_3, oof_pred_df_session_length_over_3)

0.924359062517908

In [51]:
(train_log.groupby("session_id")["yad_no"].apply(len) > 2).index

Index(['000007603d533d30453cc45d0f3d119f', '0000ca043ed437a1472c9d1d154eb49b',
       '0000d4835cf113316fe447e2f80ba1c8', '0000fcda1ae1b2f431e55a7075d1f500',
       '000104bdffaaad1a1e0a9ebacf585f33', '00011afe25c343301ee961b3b0ce2e4d',
       '000125c737df1802b6e365f93c96d3c8', '0001763050a10b21062a1304fb743fd4',
       '000178c4d4d567d4715331dd0cdab76c', '0001e6a407a85dc50ac132a5c7016bab',
       ...
       'fffe3dedeb89ae77af5caf4064113748', 'fffe40478b2163bd87ce3f45231ab67c',
       'fffe78a078a176562fa1e968b2a48a76', 'fffe8a472ae6a96c9da05a30ac3ed6c5',
       'fffe8c99c5b332190c3d4a2d6e7c5073', 'ffff2262d38abdeb247ebd591835dcc9',
       'ffff2360540745117193ecadcdc06538', 'ffff7fb4617164b2604aaf51c40bf82d',
       'ffffcd5bc19d62cad5a3815c87818d83', 'fffffa7baf370083ebcdd98f26a7e31a'],
      dtype='object', name='session_id', length=288698)

In [98]:
train_for_oof_with_yad[train_for_oof_with_yad["session_length"] == 1][100:120]

,session_id,yad_no,reserve,session_length,session_yado_nunique,session_yado_cnt,previous_1_yad_no,now_yad_no,now_yad_type,now_total_room_cnt,now_wireless_lan_flg,now_onsen_flg,now_kd_stn_5min,now_kd_bch_5min,now_kd_slp_5min,now_kd_conv_walk_5min,now_wid_cd,now_ken_cd,now_lrg_cd,now_sml_cd,previous_1_yad_type,previous_1_total_room_cnt,previous_1_wireless_lan_flg,previous_1_onsen_flg,previous_1_kd_stn_5min,previous_1_kd_bch_5min,previous_1_kd_slp_5min,previous_1_kd_conv_walk_5min,previous_1_wid_cd,previous_1_ken_cd,previous_1_lrg_cd,previous_1_sml_cd,fold,popular_yado_rate_per_fold,oof,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
189,0015e7ee70fcb94f0ebaf6d9e3099ed7,1652,0,1,1,0,13232,1652,0,127.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,2ffd60eb648dbbaa1d1ba33644813c44,0,140.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,0.000242,0.127048,0,127.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,2ffd60eb648dbbaa1d1ba33644813c44
190,0015e7ee70fcb94f0ebaf6d9e3099ed7,5767,0,1,1,0,13232,5767,0,95.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,140.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,0.000056,0.053708,0,95.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e
191,0015e7ee70fcb94f0ebaf6d9e3099ed7,6091,0,1,1,0,13232,6091,0,603.0,1.0,1,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,140.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,0.000191,0.117284,0,603.0,1.0,1,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e
192,0015e7ee70fcb94f0ebaf6d9e3099ed7,6185,0,1,1,0,13232,6185,0,182.0,NaN,1,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,140.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,0.000126,0.091584,0,182.0,NaN,1,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e
193,0015e7ee70fcb94f0ebaf6d9e3099ed7,9638,0,1,1,0,13232,9638,0,99.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,2ffd60eb648dbbaa1d1ba33644813c44,0,140.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,0.000169,0.081662,0,99.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,2ffd60eb648dbbaa1d1ba33644813c44
194,0015e7ee70fcb94f0ebaf6d9e3099ed7,9790,0,1,1,0,13232,9790,0,368.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,f2fcbd8e62872147efde0acef474e1f2,0,140.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f921c3e38b81998fbf283f01,107c7305a74c8dcc4f143de208bf7ec2,3a6cd37aa9e38fd96d9dafc2615643d0,4683b842facadc1ad7161f72220f6a3e,0,0.000074,0.028852,0,368.0,1.0,0,1.0,NaN,NaN,1.0,46e33861f9

In [41]:
# pandasで全ての列を表示する
pd.set_option("display.max_columns", None)

In [58]:
session_id = "004d95150adec9e455adcd089d193176"
train_for_oof_with_yad[train_for_oof_with_yad["session_id"] == session_id]

,session_id,yad_no,reserve,session_length,session_yado_nunique,session_yado_cnt,previous_1_yad_no,now_yad_no,now_yad_type,now_total_room_cnt,now_wireless_lan_flg,now_onsen_flg,now_kd_stn_5min,now_kd_bch_5min,now_kd_slp_5min,now_kd_conv_walk_5min,now_wid_cd,now_ken_cd,now_lrg_cd,now_sml_cd,previous_1_yad_type,previous_1_total_room_cnt,previous_1_wireless_lan_flg,previous_1_onsen_flg,previous_1_kd_stn_5min,previous_1_kd_bch_5min,previous_1_kd_slp_5min,previous_1_kd_conv_walk_5min,previous_1_wid_cd,previous_1_ken_cd,previous_1_lrg_cd,previous_1_sml_cd,fold,popular_yado_rate_per_fold,oof,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
861,004d95150adec9e455adcd089d193176,523,0,3,3,1,13198,523,0,30.0,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,0.000221,0.516190,0,30.0,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4
862,004d95150adec9e455adcd089d193176,762,0,3,3,0,13198,762,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,0.000130,0.006173,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4
863,004d95150adec9e455adcd089d193176,813,0,3,3,0,13198,813,0,52.0,NaN,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ae316115cdaf0cb1b67b65e75f260b4d,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,0.000082,0.003011,0,52.0,NaN,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ae316115cdaf0cb1b67b65e75f260b4d
864,004d95150adec9e455adcd089d193176,4536,0,3,3,0,13198,4536,0,32.0,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,0.000130,0.006173,0,32.0,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4
865,004d95150adec9e455adcd089d193176,4598,0,3,3,0,13198,4598,0,16.0,NaN,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ae316115cdaf0cb1b67b65e75f260b4d,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,0.000052,0.002413,0,16.0,NaN,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ae316115cdaf0cb1b67b65e75f260b4d
866,004d95150adec9e455adcd089d193176,6328,0,3,3,0,13198,6328,0,60.0,1.0,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ae316115cdaf0cb1b67b65e75f260b4d,0,NaN,1.0,0,NaN,NaN,NaN,1.0,c312e07b7a5d456d53a5b00910a336e1,6692a692f80687411022c08e4f5a7a00,8cf750072f8520a726ff601894c2d39e,ddd5616ecb2d2c35c8ac403db5804ef4,0,0.000234,0.004595,0,60.0,1.0,1,NaN,NaN,NaN,NaN,c312e07b7a5d456d53a5b00910a336e1,6692a

In [59]:
train_log[train_log["session_id"] == session_id]

,session_id,seq_no,yad_no
561,004d95150adec9e455adcd089d193176,0,13058
562,004d95150adec9e455adcd089d193176,1,523
563,004d95150adec9e455adcd089d193176,2,13198


In [31]:
yado[yado["yad_no"].isin([13035])]

,yad_no,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
13034,13035,0,151.0,1.0,0,NaN,NaN,NaN,NaN,e9316013ee1b03f4525fe361c46ce9c5,06a6febd7478c1aa664ed5ac87f2d560,9bbd969339a85b4a12570a1dfecad067,ffa35ab2dd7300f5c43aa7a2d174655a


In [100]:
sub_df = pd.concat([test_session, sub], axis=1)

In [70]:
# pandasで全ての行を表示する
pd.set_option("display.max_rows", None)

In [99]:
test[test["session_length"] == 1][100:120]

,session_id,yad_no,session_length,session_yado_nunique,session_yado_cnt,previous_1_yad_no,now_yad_no,now_yad_type,now_total_room_cnt,now_wireless_lan_flg,now_onsen_flg,now_kd_stn_5min,now_kd_bch_5min,now_kd_slp_5min,now_kd_conv_walk_5min,now_wid_cd,now_ken_cd,now_lrg_cd,now_sml_cd,previous_1_yad_type,previous_1_total_room_cnt,previous_1_wireless_lan_flg,previous_1_onsen_flg,previous_1_kd_stn_5min,previous_1_kd_bch_5min,previous_1_kd_slp_5min,previous_1_kd_conv_walk_5min,previous_1_wid_cd,previous_1_ken_cd,previous_1_lrg_cd,previous_1_sml_cd,popular_yado_rate_per_fold
237,00017e2a527901c9c41b1acef525d016,2179,1,1,0,4621,2179,0,252.0,1.0,0,1.0,NaN,NaN,NaN,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000201
238,00017e2a527901c9c41b1acef525d016,2180,1,1,0,4621,2180,0,217.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000298
239,00017e2a527901c9c41b1acef525d016,2234,1,1,0,4621,2234,0,4.0,1.0,0,NaN,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000003
240,00017e2a527901c9c41b1acef525d016,2293,1,1,0,4621,2293,0,53.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000066
241,00017e2a527901c9c41b1acef525d016,2308,1,1,0,4621,2308,0,134.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000069
242,00017e2a527901c9c41b1acef525d016,2319,1,1,0,4621,2319,0,NaN,1.0,0,NaN,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000294
243,00017e2a527901c9c41b1acef525d016,2355,1,1,0,4621,2355,0,373.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000301
244,00017e2a527901c9c41b1acef525d016,2448,1,1,0,4621,2448,0,177.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000090
245,00017e2a527901c9c41b1acef525d016,2500,1,1,0,4621,2500,0,163.0,1.0,0,NaN,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582c

In [103]:
session_id = "00017e2a527901c9c41b1acef525d016"

In [104]:
sub_df[sub_df["session_id"] == session_id]

,session_id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
4,00017e2a527901c9c41b1acef525d016,5713,6576,13590,5106,12524,9020,7246,3187,2893,11494


In [105]:
test_log[test_log["session_id"] == session_id]

,session_id,seq_no,yad_no
8,00017e2a527901c9c41b1acef525d016,0,4621


In [108]:
test[test["session_id"] == session_id].sort_values(
    "popular_yado_rate_per_fold", ascending=False
)

,session_id,yad_no,session_length,session_yado_nunique,session_yado_cnt,previous_1_yad_no,now_yad_no,now_yad_type,now_total_room_cnt,now_wireless_lan_flg,now_onsen_flg,now_kd_stn_5min,now_kd_bch_5min,now_kd_slp_5min,now_kd_conv_walk_5min,now_wid_cd,now_ken_cd,now_lrg_cd,now_sml_cd,previous_1_yad_type,previous_1_total_room_cnt,previous_1_wireless_lan_flg,previous_1_onsen_flg,previous_1_kd_stn_5min,previous_1_kd_bch_5min,previous_1_kd_slp_5min,previous_1_kd_conv_walk_5min,previous_1_wid_cd,previous_1_ken_cd,previous_1_lrg_cd,previous_1_sml_cd,popular_yado_rate_per_fold
342,00017e2a527901c9c41b1acef525d016,9020,1,1,0,4621,9020,0,235.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000845
293,00017e2a527901c9c41b1acef525d016,5713,1,1,0,4621,5713,0,504.0,NaN,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000703
400,00017e2a527901c9c41b1acef525d016,12524,1,1,0,4621,12524,0,409.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000693
305,00017e2a527901c9c41b1acef525d016,6576,1,1,0,4621,6576,0,310.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000679
315,00017e2a527901c9c41b1acef525d016,7246,1,1,0,4621,7246,0,186.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000623
417,00017e2a527901c9c41b1acef525d016,13590,1,1,0,4621,13590,0,96.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000620
282,00017e2a527901c9c41b1acef525d016,5106,1,1,0,4621,5106,0,120.0,1.0,0,NaN,NaN,NaN,NaN,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000606
259,00017e2a527901c9c41b1acef525d016,3187,1,1,0,4621,3187,0,211.0,1.0,0,NaN,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,084c46af580a4871ae2a4921addf6e8e,0.000596
352,00017e2a527901c9c41b1acef525d016,9623,1,1,0,4621,9623,0,245.0,1.0,0,NaN,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e30155582cf80b1d655986,7763c74e2efa67a522125d9d3d7dde25,a0549a1384a56d5424439c07d0239850,0,174.0,1.0,0,1.0,NaN,NaN,1.0,e9316013ee1b03f4525fe361c46ce9c5,66c4d01ad8e3